# 第四章 &nbsp; &nbsp; word2vec的高速化 
上一章我们学习了 word2vec 的机制，并实现了 CBOW 模型。因为 CBOW 模型是一个简单的 2 层神经网络，所以实现起来比较简单。但是，目前的实现存在几个问题，其中最大的问题是，随着语料库中处理的词汇量的增加，计算量也随之增加。实际上，当词汇量达到一定程度之后，上一章的 CBOW 模型的计算就会花费过多的时间。

因此，本章将重点放在 word2vec 的加速上，来改善 word2vec。具体而言，我们将对上一章中简单的 word2vec 进行两点改进：引入名为 Embedding 层的新层，以及引入名为 Negative Sampling 的新损失函数。这样一来，我们就能够完成一个“真正的”word2vec。完成这个真正的 word2vec 后，我们将在 PTB 数据集（一个大小比较实用的语料库）上进行学习，并实际评估所获得的单词的分布式表示的优劣。

### 4.1 word2vec的改进
我们先复习一下上一章的内容。在上一章中，我们实现了下图中的 CBOW 模型。

<img src="./fig/CBOW_model.png" alt="CBOW_model" style="display: block; margin: 0 auto;">

如图所示，上一章的 CBOW 模型接收拥有 2 个单词的上下文，并基于它们预测 1 个单词（目标词）。此时，通过输入层和输入侧权重（$\boldsymbol{W}_{\text{in}}$）之间的矩阵乘积计算中间层，通过中间层和输出侧权重（$\boldsymbol{W}_{\text{out}}$）之间的矩阵乘积计算每个单词的得分。这些得分经过 Softmax 函数转化后，得到每个单词的出现概率。通过将这些概率与正确解标签进行比较（更确切地说，使用交叉熵误差函数），从而计算出损失。

在上一章中，我们限定了上下文的窗口大小为1。这相当于只将目标词的前一个和后一个单词作为上下文。本章我们将给模型新增一个功能，使之能够处理任意窗口大小的上下文。

上图中的 CBOW 模型在处理小型语料库时问题不大。实际上，图中处理的词汇量一共只有 7 个，这个规模自然毫无问题。不过在处理大规模语料库时，这个模型就存在多个问题了。为了指出这些问题，这里我们考虑一个例子。假设词汇量有 100 万个，CBOW 模型的中间层神经元有 100 个，此时 word2vec 进行的处理如下图所示。

<img src="./fig/CBOW_problem.png" alt="CBOW_problem" style="display: block; margin: 0 auto;">

如图所示，输入层和输出层存在 100 万个神经元。在如此多的神经元的情况下，中间的计算过程需要很长时间。具体来说，以下两个地方的计算会出现瓶颈。

- 输入层的 one-hot 表示和权重矩阵 $\boldsymbol{W}_{\text{in}}$ 的乘积
- 中间层和权重矩阵 $\boldsymbol{W}_{\text{out}}$ 的乘积以及 Softmax 层的计算

第 1 个问题与输入层的 one-hot 表示有关。这是因为我们用 one-hot 表示来处理单词，随着词汇量的增加，one-hot 表示的向量大小也会增加。比如，在词汇量有 100 万个的情况下，仅 one-hot 表示本身就需要占用 100 万个元素的内存大小。此外，还需要计算 one-hot 表示和权重矩阵 $\boldsymbol{W}_{\text{in}}$ 的乘积，这也要花费大量的计算资源。关于这个问题，我们会在本节中通过引入新的 Embedding 层来解决。

第 2 个问题是中间层之后的计算。首先，中间层和权重矩阵 $\boldsymbol{W}_{\text{out}}$ 的乘积需要大量的计算。其次，随着词汇量的增加，Softmax 层的计算量也会增加。关于这些问题，我们将在下小节通过引入 Negative Sampling 这一新的损失函数来解决。下面就让我们通过改进来消除这两个瓶颈。

## Embedding层
在上一章的 word2vec 实现中，我们将单词转化为了 one-hot 表示，并将其输入了 MatMul 层，在 MatMul 层中计算了该 one-hot 表示和权重矩阵的乘积。这里，我们来考虑词汇量是 100 万个的情况。假设中间层的神经元个数是 100，则 MatMul 层中的矩阵乘积可画成下图。

<img src="./fig/matmul_one_hot.png" alt="matmul_one_hot" style="display: block; margin: 0 auto;">

如图所示，如果语料库的词汇量有 100 万个，则单词的 one-hot 表示的维数也会是 100 万，我们需要计算这个巨大向量和权重矩阵的乘积。但是，图中所做的无非是将矩阵的某个特定的行取出来。因此，直觉上将单词转化为 one-hot 向量的处理和 MatMul 层中的矩阵乘法似乎没有必要。

现在，我们创建一个从权重参数中抽取“单词 ID 对应行（向量）”的层，这里我们称之为 Embedding 层。顺便说一句，Embedding 来自“词嵌入”（word embedding）这一术语。也就是说，在这个 Embedding 层存放词嵌入（分布式表示）。

在自然语言处理领域，单词的密集向量表示称为<strong>词嵌入</strong>（word embedding）或者单词的<strong>分布式表示</strong>（distributed representation）。过去，将基于计数的方法获得的单词向量称为 distributional representation，将使用神经网络的基于推理的方法获得的单词向量称为 distributed representation。不过，中文里二者都译为“分布式表示”。

## Embedding层的实现
从矩阵中取出某一行的处理是很容易实现的。这里，假设权重 `W` 是 NumPy 的二维数组。如果要从这个权重中取出某个特定的行，只需写 `W[2]` 或者 `W[5]`。用 Python 代码来实现，如下所示。

In [1]:
import numpy as np

W = np.arange(21).reshape(7, 3) # 7行3列的二维数组，arange表示从0开始的等差数列
print(W) # 输出W
print(W.shape) # 输出W的形状

print(W[2]) # 输出W的第2行
print(W[5]) # 输出W的第5行

idx = np.array([1, 0, 3, 0]) # 索引数组
print(W[idx]) # 输出W的第1,0,3,0行

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
(7, 3)
[6 7 8]
[15 16 17]
[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


下面，我们来实现 Embedding 层的 ``forward()`` 方法。参照之前的例子，实现如下所示。

In [2]:
class Embedding:
    def __init__(self, W):
        self.params = [W] # 将权重保存在params属性中
        self.grads = [np.zeros_like(W)] # 将梯度保存在grads属性中
        self.idx = None # 初始化idx属性

    def forward(self, idx):
        W, = self.params # 从params属性中取出权重
        self.idx = idx # 将输入的索引保存到idx属性中
        out = W[idx] # 根据索引取出对应的行
        return out # 返回取出的行

根据本书的代码规范，使用 `params` 和 `grads` 作为成员变量，并在成员变量 `idx` 中以数组的形式保存需要提取的行的索引（单词 ID）。

接下来，我们考虑反向传播。Embedding 层的正向传播只是从权重矩阵 `W` 中提取特定的行，并将该特定行的神经元原样传给下一层。因此，在反向传播时，从上一层（输出侧的层）传过来的梯度将原样传给下一层（输入侧的层）。不过，从上一层传来的梯度会被应用到权重梯度 `dW` 的特定行（`idx`），如图所示。

<img src="./fig/embedding.png" alt="embedding" style="display: block; margin: 0 auto;">

基于以上内容，我们来实现 ``backward()``，代码如下所示。

In [3]:
def backward(self, dout):
    dW, = self.grads # 从grads属性中取出梯度
    dW[...] = 0 # 将梯度清零
    dW[self.idx] = dout # 将上游传来的梯度写入对应的行
    return None # 返回None

这里，取出权重梯度 `dW`，通过 `dW[...] = 0` 将 `dW` 的元素设为 0（并不是将 `dW` 设为 0，而是保持 `dW` 的形状不变，将它的元素设为 0）。然后，将上一层传来的梯度 `dout` 写入 `idx` 指定的行。

这里创建了和权重W相同大小的矩阵dW，并将梯度写入了dW对应的行。但是，我们最终想做的事情是更新权重W，所以没有必要特意创建dW（大小与W相同）。相反，只需把需要更新的行号（idx）及其对应的梯度（dout）保存下来，就可以更新权重（W）的特定行。但是，这里为了兼容已经实现的优化器类（Optimizer），所以写成了现在的样子。

实际上，在刚才的 `backward()` 的实现中，存在一个问题，这一问题发生在 `idx` 的元素出现重复时。比如，当 `idx` 为 `[0, 2, 0, 4]` 时，就会发生下图中的问题。

<img src="./fig/embedding_problem.png" alt="embedding_problem" style="display: block; margin: 0 auto;">

如图所示，我们将 `dh` 各行的值写入 `dW` 中 `idx` 指定的位置。在这种情况下，`dW` 的第 0 行会被写入两次。这样一来，其中某个值就会被覆盖掉。

为了解决这个重复问题，需要进行“加法”，而不是“写入”（请读者考虑一下为什么是加法）。也就是说，应该把 `dh` 各行的值累加到 `dW` 的对应行中。下面，我们来实现正确的反向传播。

In [4]:
def backward(self, dout):
        dW, = self.grads # 从grads属性中取出梯度
        dW[...] = 0 # 将梯度初始化为0
        np.add.at(dW, self.idx, dout) # 将dout累加到对应的行
        return None # 返回None

关于 Embedding 层的实现就介绍到这里。现在，我们可以将 word2vec（CBOW 模型）的实现中的输入侧的 MatMul 层换成 Embedding 层。这样一来，既能减少内存使用量，又能避免不必要的计算。

In [5]:
class Embedding:
    def __init__(self, W):
        self.params = [W] # 将权重保存在params属性中
        self.grads = [np.zeros_like(W)] # 将梯度保存在grads属性中
        self.idx = None # 初始化idx属性

    def forward(self, idx):
        W, = self.params # 从params属性中取出权重
        self.idx = idx # 将输入的索引保存到idx属性中
        out = W[idx] # 根据索引取出对应的行
        return out # 返回取出的行

    def backward(self, dout):
        dW, = self.grads # 从grads属性中取出梯度
        dW[...] = 0 # 将梯度清零
        np.add.at(dW, self.idx, dout) # 根据索引将上游传下来的梯度加到对应的行
        return None # Embedding层没有输入，所以返回None

## word2vec的改进
下面，我们来进行 word2vec 的第 2 个改进。如前所述，word2vec 的另一个瓶颈在于中间层之后的处理，即矩阵乘积和 Softmax 层的计算。本节的目标就是解决这个瓶颈。这里，我们将采用名为**负采样**（negative sampling）的方法作为解决方案。使用 Negative Sampling 替代 Softmax，无论词汇量有多大，都可以使计算量保持较低或恒定。

本节的内容有些复杂，特别是实现方面会有点“纠结”。因此，我们会一个知识点一个知识点地确认，一步一步地前进。

## 中间层之后的计算问题
为了指出中间层之后的计算问题，和上一节一样，我们来考虑词汇量为 100 万个、中间层的神经元个数为 100 个的 word2vec（CBOW 模型）。此时，word2vec 进行的处理如图所示。

<img src="./fig/CBOW_problem.png" alt="CBOW_problem" style="display: block; margin: 0 auto;">

如图所示，输入层和输出层有 100 万个神经元。在上一节中，通过引入 Embedding 层，节省了输入层中不必要的计算。剩下的问题就是中间层之后的处理。此时，在以下两个地方需要很多计算时间。

- 中间层的神经元和权重矩阵（$\boldsymbol{W}_{\text{out}}$）的乘积
- Softmax 层的计算

第 1 个问题在于巨大的矩阵乘积计算。在上面的例子中，中间层向量的大小是 100，权重矩阵的大小是 $100 \times 1000000$，如此巨大的矩阵乘积计算需要大量时间（也需要大量内存）。此外，因为反向传播时也要进行同样的计算，所以很有必要将矩阵乘积计算“轻量化”。

其次，Softmax 也会发生同样的问题。换句话说，随着词汇量的增加，Softmax 的计算量也会增加。观察 Softmax 的公式，就可以清楚地看出这一点。

$$y_k = \frac{\exp(s_k)}{\sum_{i=1}^{1000000} \exp(s_i)} \tag{4.1}$$

式 (4.1) 是第 $k$ 个元素（单词）的 Softmax 的计算式（各个元素的得分为 $s_1, s_2, \cdots$）。因为假定词汇量是 100 万个，所以式 (4.1) 的分母需要进行 100 万次的 $\exp$ 计算。这个计算也与词汇量成正比，因此，需要一个可以替代 Softmax 的“轻量”的计算。

### 从多分类到二分类
下面，我们来解释一下负采样。这个方法的关键思想在于二分类（binary classification），更准确地说，是用二分类拟合多分类（multiclass classification），这是理解负采样的重点。

到目前为止，我们处理的都是多分类问题。拿刚才的例子来说，我们把它看作了从 100 万个单词中选择 1 个正确单词的任务。那么，可不可以将这个问题处理成二分类问题呢？更确切地说，我们是否可以用二分类问题来拟合这个多分类问题呢？

二分类处理的是答案为 “Yes/No” 的问题。诸如，“这个数字是7吗？”“这是猫吗？”“目标词是say吗？” 等，这些问题都可以用 “Yes/No” 来回答。

到目前为止，我们已经做到了当给定上下文时，以较高的概率预测出作为正确解的单词。比如，当给定 you 和 goodbye 时，使神经网络预测出单词 say 的概率最高。如果学习进展顺利，神经网络就可以进行正确的预测。换句话说，对于“当上下文是 you 和 goodbye 时，目标词是什么？”这个问题，神经网络可以给出正确答案。

现在，我们来考虑如何将多分类问题转化为二分类问题。为此，我们先考察一个可以用 “Yes/No” 来回答的问题。比如，让神经网络来回答 “当上下文是 you 和 goodbye 时，目标词是 say 吗？” 这个问题，这时输出层只需要一个神经元即可。可以认为输出层的神经元输出的是 say 的得分。

那么，此时 CBOW 模型进行什么样的处理呢？用图来表示的话，可以画出下图。

<img src="./fig/target_network.png" alt="target_network" style="display: block; margin: 0 auto;">

如图所示，输出层的神经元仅有一个。因此，要计算中间层和输出侧的权重矩阵的乘积，只需要提取 say 对应的列（单词向量），并用它与中间层的神经元计算内积即可。这个计算的详细过程如下图所示。

<img src="./fig/say_dot.png" alt="say_dot" style="display: block; margin: 0 auto;">

如图所示，输出侧的权重 $\boldsymbol{W}_{\text{out}}$ 中保存了各个单词 ID 对应的单词向量。此处，我们提取 say 这个单词向量，再求这个向量和中间层神经元的内积，这就是最终的得分。

到目前为止，输出层是以全部单词为对象进行计算的。这里，我们仅关注单词say，计算它的得分。然后，使用sigmoid函数将其转化为概率。

### 4.2.3 sigmoid函数和交叉熵误差
要使用神经网络解决二分类问题，需要使用 sigmoid 函数将得分转化为概率。为了求损失，我们使用交叉熵误差作为损失函数。这些都是二分类神经网络的老套路。

在多分类的情况下，输出层使用Softmax函数将得分转化为概率，损失函数使用交叉熵误差。在二分类的情况下，输出层使用sigmoid函数，损失函数也使用交叉熵误差。

这里我们先回顾一下 sigmoid 函数，如下式所示：

$$y = \frac{1}{1 + \exp(-x)} \tag{4.2}$$

式 (4.2) 的图像如下图中的右图所示。从图中可以看出，sigmoid 函数呈 S 形，输入值 $x$ 被转化为 0 到 1 之间的实数。这里的要点是，sigmoid 函数的输出 $y$ 可以解释为概率。

另外，与 sigmoid 函数相关的 Sigmoid 层已经实现好了，如图中的左图所示。

<img src="./fig/sigmoid.png" alt="sigmoid" style="display: block; margin: 0 auto;">

通过 sigmoid 函数得到概率 $y$ 后，可以由概率 $y$ 计算损失。与多分类一样，用于 sigmoid 函数的损失函数也是交叉熵误差，其数学式如下所示：

$$L = -\left(t \log y + (1 - t) \log (1 - y)\right) \tag{4.3}$$

其中，$y$ 是 sigmoid 函数的输出，$t$ 是正确解标签，取值为 0 或 1：取值为 1 时表示正确解是 “Yes”；取值为 0 时表示正确解是 “No”。因此，当 $t$ 为 1 时，输出 $-\log y$；当 $t$ 为 0 时，输出 $-\log (1 - y)$。

二分类和多分类的损失函数均为交叉熵误差，其数学式分别为式(4.3)和式(1.7)。不过，它们只是写法不同而已，实际上表示的内容是一样的。确切地说，在多分类的情况下，如果输出层只有两个神经元，则式(1.7)和二分类的式(4.3)是完全一致的。因此，Sigmoid with Loss层的实现只要在Softmax with Loss层的基础上稍加改动即可。

下面，我们用图来表示 Sigmoid 层和 Cross Entropy Error 层，如图所示。

<img src="./fig/sigmoid_with_loss.png" alt="sigmoid_with_loss" style="display: block; margin: 0 auto;">

图中值得注意的是反向传播的 $y - t$ 这个值。$y$ 是神经网络输出的概率，$t$ 是正确解标签，$y - t$ 正好是这两个值的差。这意味着，当正确解标签是 1 时，如果 $y$ 尽可能地接近 1（100%），误差将很小。反过来，如果 $y$ 远离 1，误差将增大。随后，这个误差向前面的层传播，当误差大时，模型学习得多；当误差小时，模型学习得少。

sigmoid 函数和交叉熵误差的组合产生了 $y - t$ 这样一个漂亮的结果。同样地，Softmax 函数和交叉熵误差的组合，或者恒等函数和均方误差的组合也会在反向传播时传播 $y - t$。

### 多分类到二分类的实现
下面，我们从实现的角度把之前讲的内容整理一下。前面我们处理了多分类问题，在输出层使用了与词汇量同等数量的神经元，并将它们传给了 Softmax 层。如果把重点放在“层”和“计算”上，则此时的神经网络可以画成下图。

<img src="./fig/CBOW_embedding.png" alt="CBOW_embedding" style="display: block; margin: 0 auto;">

图中展示了上下文是 you 和 goodbye、作为正确解的目标词是 say 的例子（假定 you 的单词 ID 是 0，say 的单词 ID 是 1，goodbye 的单词 ID 是 2）。在输入层中，为了提取单词 ID 对应的分布式表示，使用了 Embedding 层。

上一节，我们实现了 Embedding 层，该层提取单词 ID 对应的分布式表示（单词向量）。以前我们用的是 MatMul 层。

现在，我们将图中的神经网络转化成进行二分类的神经网络，网络结构如下图所示。

<img src="./fig/embedding_binary.png" alt="embedding_binary" style="display: block; margin: 0 auto;">

这里，将中间层的神经元记为 $\boldsymbol{h}$，并计算它与输出侧权重 $\boldsymbol{W}_{\text{out}}$ 中的单词 say 对应的单词向量的内积。然后，将其输出输入 Sigmoid with Loss 层，得到最终的损失。

在上图中，向 Sigmoid with Loss 层输入正确解标签 1，这意味着现在正在处理的问题的答案是 “Yes”。当答案是 “No” 时，向 Sigmoid with Loss 层输入 0。

为了便于理解后面的内容，我们把图的后半部分进一步简化。为此，我们引入 Embedding Dot 层，该层将图中的 Embedding 层和 dot 运算（内积）合并起来处理。使用这个层，上图的后半部分可以画成下图。

<img src="./fig/embedding_dot.png" alt="embedding_dot" style="display: block; margin: 0 auto;">

中间层的神经元 $\boldsymbol{h}$ 流经 Embedding Dot 层，传给 Sigmoid with Loss 层。从图中可以看出，使用 Embedding Dot 层之后，中间层之后的处理被简化了。

下面，我们简单地看一下 Embedding Dot 层的实现，这里我们将这个层实现为 `EmbeddingDot` 类。


In [6]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W) # 创建Embedding层
        self.params = self.embed.params # 将Embedding层的参数赋值给params属性
        self.grads = self.embed.grads # 将Embedding层的梯度赋值给grads属性
        self.cache = None # 初始化cache属性

    def forward(self, h, idx):
        target_W = self.embed.forward(idx) # 通过Embedding层获取目标词的权重
        out = np.sum(h * target_W, axis=1) # 前向传播，计算点积，得到输出层
        self.cache = (h, target_W) # 保存中间变量以备反向传播使用
        return out # 返回点积结果

    def backward(self, dout):
        h, target_W = self.cache # 从cache中取出中间变量
        dH = dout[:, np.newaxis] * target_W # 反向传播，计算上游梯度对h的影响，dout[:, np.newaxis] 会把 [n] 变成 [n, 1]（增加一个维度，用于广播）
        self.embed.backward(dH) # 反向传播到Embedding层
        return None # 返回None

---

在这段代码中，`EmbeddingDot` 类能够与内部的 `Embedding` 层共享参数和梯度，核心原因在于 **Python 中对象引用的特性** 以及 **神经网络参数更新的设计逻辑**。我们可以从两个层面具体理解：


### 1. 从代码实现看：参数是“引用”而非“复制”
在 `__init__` 方法中：
```python
self.embed = Embedding(W)  # 创建Embedding层实例
self.params = self.embed.params  # 直接赋值，不是复制
self.grads = self.embed.grads  # 直接赋值，不是复制
```
这里的 `self.params` 和 `self.embed.params` 指向 **同一个内存地址**（即对同一个对象的引用），而非两份独立的参数。梯度 `self.grads` 与 `self.embed.grads` 也是同样的道理。

这意味着：
- 当 `Embedding` 层的 `params`（如词嵌入矩阵 `W`）被修改时，`self.params` 会同步变化（因为它们是同一个对象）。
- 反向传播时，`Embedding` 层计算出的梯度会直接存储在 `self.embed.grads` 中，而 `self.grads` 作为引用，自然也能访问到这些梯度。


### 2. 从神经网络逻辑看：参数共享是刻意设计的需求
在词嵌入模型（如 Word2Vec）中，`Embedding` 层的核心参数是 **词向量矩阵 `W`**（形状为 `[词汇表大小, 嵌入维度]`）。这个矩阵是模型需要学习的核心知识：每个行向量代表一个词的嵌入表示。

`EmbeddingDot` 类的作用是“使用”这些词向量进行计算（计算上下文与目标词的相似度），但它 **不需要独立维护另一套词向量**。原因是：
- 模型的目标是学习出一套统一的词向量（`W`），所有用到词向量的地方都应该共享这一套参数，否则会导致学习目标不一致（比如同一个词在不同地方有不同的向量）。
- 反向传播时，梯度需要最终汇总到 `W` 上进行更新。如果 `EmbeddingDot` 单独维护参数，梯度就无法正确传递到原始的 `W`，导致参数无法更新。


### 举个通俗的例子
假设 `Embedding` 层的 `params` 是一个笔记本，记录了所有词的向量。`EmbeddingDot` 的 `self.params` 并不是复印了一本新笔记本，而是拿到了这本笔记本的“共享权限”。

- 当 `Embedding` 层修改笔记本内容（比如更新某个词的向量），`EmbeddingDot` 看到的也会是修改后的内容。
- 当 `EmbeddingDot` 需要用到词向量时，直接查阅这本共享笔记本即可，无需自己再抄一份。


### 总结
参数和梯度的“共享”本质上是通过 **对象引用** 实现的，而这种设计是为了保证：
1. 整个模型中词向量的唯一性（一套词向量供所有模块使用）。
2. 反向传播时梯度能正确传递到核心参数 `W`，确保参数可以被有效更新。

这是神经网络模块化设计中的常见技巧：通过共享参数减少冗余，同时保证模型逻辑的一致性。

---

`EmbeddingDot` 类共有 4 个成员变量：`embed`、`params`、`grads` 和 `cache`。根据本书的代码规范，`params` 保存参数，`grads` 保存梯度。另外，作为缓存，`embed` 保存 `Embedding` 层，`cache` 保存正向传播时的计算结果。

正向传播的 `forward(h, idx)` 方法的参数接收中间层的神经元（`h`）和单词 ID 的 NumPy 数组（`idx`）。这里，`idx` 是单词 ID 列表，这是因为我们假定了对数据进行 mini-batch 处理。

在上面的代码中，`forward()` 方法首先调用 `Embedding` 层的 `forward(idx)` 方法，然后通过 `np.sum(self.target_w * h, axis=1)` 计算内积。通过观察具体值来理解这个实现会比较快，如图所示。

<img src="./fig/embedding_dot_example.png" alt="embedding_dot_example" style="display: block; margin: 0 auto;">

如图所示，准备适当的 `W`、`h` 和 `idx`。这里，`idx` 是 `[0, 3, 1]`，这个例子表示 mini-batch 一并处理 3 笔数据。因为 `idx` 是 `[0, 3, 1]`，所以 `target_w` 将提取出 `W` 的第 0 行、第 3 行和第 1 行。另外，`target_w * h` 计算对应元素的乘积（NumPy 的 “*” 计算对应元素的乘积）。然后，对结果逐行（`axis=1`）进行求和，得到最终的结果 `out`。

以上就是对 Embedding Dot 层的正向传播的介绍。反向传播以相反的顺序传播梯度，这里我们省略对其实现的说明（并不是特别难，请大家自己思考）。

## 负采样
至此，我们成功地把要解决的问题从多分类问题转化成了二分类问题。但是，这样问题就被解决了吗？很遗憾，事实并非如此。因为我们目前仅学习了正例（正确答案），还不确定负例（错误答案）会有怎样的结果。

现在，我们再来思考一下之前的例子。在之前的例子中，上下文是 you 和 goodbye，目标词是 say。我们到目前为止只是对正例 say 进行了二分类，如果此时模型有“好的权重”，则 Sigmoid 层的输出（概率）将接近 1。用计算图来表示此时的处理，如图所示。

<img src="./fig/CBOW_middle_progress.png" alt="CBOW_middle_progress" style="display: block; margin: 0 auto;">

当前的神经网络只是学习了正例 say，但是对 say 之外的负例一无所知。而我们真正要做的事情是，对于正例（say），使 Sigmoid 层的输出接近 1；对于负例（say 以外的单词），使 Sigmoid 层的输出接近 0。用图来表示，如图所示。

<img src="./fig/CBOW_positive_negative.png" alt="CBOW_positive_negative" style="display: block; margin: 0 auto;">

比如，当上下文是 you 和 goodbye 时，我们希望目标词是 hello（错误答案）的概率较低。在图中，目标词是 hello 的概率为 0.021（2.1%），我们要求的就是这种能使输出接近 0 的权重。

为了把多分类问题处理为二分类问题，对于“正确答案”（正例）和“错误答案”（负例），都需要能够正确地进行分类（二分类）。因此，需要同时考虑正例和负例。

那么，我们需要以所有的负例为对象进行学习吗？答案显然是 “No”。如果以所有的负例为对象，词汇量将暴增至无法处理（更何况本章的目的本来就是解决词汇量增加的问题）。为此，作为一种近似方法，我们将选择若干个（5 个或者 10 个）负例（如何选择将在下文介绍）。也就是说，只使用少数负例。这就是负采样方法的含义。

总而言之，负采样方法既可以求将正例作为目标词时的损失，同时也可以采样（选出）若干个负例，对这些负例求损失。然后，将这些数据（正例和采样出来的负例）的损失加起来，将其结果作为最终的损失。

下面，让我们结合具体的例子来说明。这里使用与之前相同的例子（正例目标词是 say）。假设选取 2 个负例目标词 hello 和 i，此时，如果我们只关注 CBOW 模型的中间层之后的部分，则负采样的计算图如图所示。

<img src="./fig/negative_sampling_example.png" alt="negative_sampling_example" style="display: block; margin: 0 auto;">

图中需要注意的是对正例和负例的处理。正例（say）和之前一样，向 Sigmoid with Loss 层输入正确解标签 1；而因为负例（hello 和 i）是错误答案，所以要向 Sigmoid with Loss 层输入正确解标签 0。此后，将各个数据的损失相加，作为最终损失输出。

## 负采样的采样方法
下面我们来看一下如何抽取负例。关于这一点，基于语料库的统计数据进行采样的方法比随机抽样要好。具体来说，就是**让语料库中经常出现的单词容易被抽到**，让语料库中不经常出现的单词难以被抽到。

基于语料库中单词使用频率的采样方法会先计算语料库中各个单词的出现次数，并将其表示为“概率分布”，然后使用这个概率分布对单词进行采样。

<img src="./fig/negative_sampling.png" alt="negative_sampling" style="display: block; margin: 0 auto;">

基于语料库中各个单词的出现次数求出概率分布后，只需根据这个概率分布进行采样就可以了。通过根据概率分布进行采样，语料库中经常出现的单词将容易被抽到，而“稀有单词”将难以被抽到。

负采样应当尽可能多地覆盖负例单词，但是考虑到计算的复杂度，有必要将负例限定在较小范围内（5个或者10个）。这里，如果只选择稀有单词作为负例会怎样呢？结果会很糟糕。因为在现实问题中，稀有单词基本上不会出现。也就是说，处理稀有单词的重要性较低。相反，处理好高频单词才能获得更好的结果。

下面，我们使用 Python 来说明基于概率分布的采样。为此，可以使用 NumPy 的 `np.random.choice()` 方法。这里我们结合几个具体的例子来看一下这个方法的用法。


In [7]:
import numpy as np 

# 随机选择一个数字
print(np.random.choice(10))

# 随机选择一个单词
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
print(np.random.choice(words))

# 有放回采样5次
print(np.random.choice(10, size=5))

# 无放回采样5次
print(np.random.choice(10, size=5, replace=False))

# 基于概率分布进行采样
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

9
hello
[9 8 0 0 3]
[6 4 8 7 5]
you


如上所示，`np.random.choice()` 可以用于随机抽样。如果指定 `size` 参数，将执行多次采样。如果指定 `replace=False`，将进行无放回采样。通过给参数 `p` 指定表示概率分布的列表，将进行基于概率分布的采样。剩下的就是使用这个函数抽取负例。

word2vec 中提出的负采样对刚才的概率分布增加了一个步骤。如式 (4.4) 所示，对原来的概率分布取 0.75 次方。

$$P'(w_i) = \frac{P(w_i)^{0.75}}{\sum_{j=1}^{n} P(w_j)^{0.75}} \tag{4.4}$$

这里，$P(w_i)$ 表示第 $i$ 个单词的概率。式 (4.4) 只是对原来的概率分布的各个元素取 0.75 次方。不过，为了使变换后的概率总和仍为 1，分母需要变成“变换后的概率分布的总和”。

那么，为什么我们要进行式 (4.4) 的变换呢？这是为了防止低频单词被忽略。更准确地说，通过取 0.75 次方，低频单词的概率将稍微变高。我们来看一个具体例子，如下所示。

In [8]:
import numpy as np
# 原始概率分布
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75) # 对概率分布的每个元素取0.75次方
new_p /= new_p.sum() # 归一化，使概率和为1
print(new_p) # 输出新的概率分布

[0.64196878 0.33150408 0.02652714]


根据这个例子，变换前概率为 0.01（1%）的元素，变换后为 0.026…（2.6…%）。通过这种方式，取 0.75 次方作为一种补救措施，使得低频单词稍微更容易被抽到。此外，0.75 这个值并没有什么理论依据，也可以设置成 0.75 以外的值。

如上所示，负采样从语料库生成单词的概率分布，在取其 0.75 次方之后，再使用之前的 `np.random.choice()` 对负例进行采样。本书中将这些处理实现为了 `UnigramSampler` 类。这里仅简单说明 `UnigramSampler` 类的使用方法。

unigram 是 “1 个（连续）单词” 的意思。同样地，bigram 是 “2 个连续单词” 的意思，trigram 是 “3 个连续单词” 的意思。这里使用 UnigramSampler 这个名字，是因为我们以 1 个单词为对象创建概率分布。如果是 bigram，则以 ('you', 'say')、('you', 'goodbye')……这样的 2 个单词的组合为对象创建概率分布。

在进行初始化时，`UnigramSampler` 类取 3 个参数，分别是单词 ID 列表格式的 `corpus`、对概率分布取的次方值 `power`（默认值是 0.75）和负例的采样个数 `sample_size`。`UnigramSampler` 类有 `get_negative_sample(target)` 方法，该方法以参数 `target` 指定的单词 ID 为正例，对其他的单词 ID 进行采样。下面，我们来看一个简单的 `UnigramSampler` 类的使用示例。

In [9]:
import sys
sys.path.append('..') # 添加上级目录到系统路径中
from common.np import *  # 导入common文件夹中的np模块
from common.layers import Embedding, SigmoidWithLoss
import collections

# 定义EmbeddingDot类
class EmbeddingDot: 
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx) # 通过Embedding层获取目标词的权重
        out = np.sum(target_W * h, axis=1) # 前向传播，计算点积，得到输出层
        self.cache = (h, target_W) # 保存中间变量以备反向传播使用
        return out

    def backward(self, dout):
        h, target_W = self.cache # 从cache中取出中间变量
        dout = dout.reshape(dout.shape[0], 1) # 将dout的形状变为 (batch_size, 1)
        dtarget_W = dout * h # 计算上游梯度对target_W的影响，形状为 (batch_size, embedding_size)
        self.embed.backward(dtarget_W) # 反向传播到Embedding层
        dh = dout * target_W # 计算上游梯度对h的影响，形状为 (batch_size, embedding_size)
        return dh

# 负采样
class UnigramSampler: 
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size # 每个正例要采样的负例数量
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter() # 统计每个单词出现的次数，collections.Counter是一个字典子类
        for word_id in corpus:
            counts[word_id] += 1 # 统计词频

        vocab_size = len(counts) # 词汇表大小，即不同单词的数量
        self.vocab_size = vocab_size 

        self.word_p = np.zeros(vocab_size) # 初始化词频数组
        for i in range(vocab_size):
            self.word_p[i] = counts[i] # 将词频写入数组

        self.word_p = np.power(self.word_p, power) # 对词频取power次方
        self.word_p /= np.sum(self.word_p) # 归一化，使概率和为1

    # 采样负例
    def get_negative_sample(self, target):
        batch_size = target.shape[0] # 批量大小

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32) # 初始化负例数组

            for i in range(batch_size):
                p = self.word_p.copy() # 复制概率分布
                target_idx = target[i] # 目标词的索引
                p[target_idx] = 0 # 将目标词的概率设为0，避免采样到目标词
                p /= p.sum() # 重新归一化
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p) # 采样负例
        else:
            # 在用GPU(cupy）计算时，优先速度
            # 有时目标词存在于负例中，体现了深度学习中 "在可接受的精度损失下追求计算效率" 的常见思路
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

# 负采样损失函数
class NegativeSamplingLoss: 
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size # 每个正例要采样的负例数量
        self.sampler = UnigramSampler(corpus, power, sample_size) # 创建负采样器
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] # 创建损失层，正例1个，负例sample_size个
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)] # 创建EmbeddingDot层，正例1个，负例sample_size个

        self.params, self.grads = [], [] # 初始化参数和梯度列表
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0] # 批量大小
        negative_sample = self.sampler.get_negative_sample(target) # 采样负例

        # 正例的正向传播
        score = self.embed_dot_layers[0].forward(h, target) # 计算正例的得分
        correct_label = np.ones(batch_size, dtype=np.int32) # 正例的标签为1，np.ones创建一个全1数组，大小为(batch_size,)
        loss = self.loss_layers[0].forward(score, correct_label) # 计算正例的损失

        # 负例的正向传播
        negative_label = np.zeros(batch_size, dtype=np.int32) # 负例的标签为0
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i] # 取出第i个负例
            score = self.embed_dot_layers[1 + i].forward(h, negative_target) # 计算负例的得分
            loss += self.loss_layers[1 + i].forward(score, negative_label) # 累加负例的损失

        return loss

    def backward(self, dout=1):
        dh = 0 # 初始化h的梯度
        # zip(...)：将两个列表按顺序打包成元组（如(正例损失层, 正例EmbeddingDot层)、(负例1损失层, 负例1 EmbeddingDot层)等），循环一次处理一对层。
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers): 
            dscore = l0.backward(dout) # 计算损失层的梯度
            dh += l1.backward(dscore) # 计算EmbeddingDot层的梯度，并累加到h的梯度

        return dh

## 改进版word2vec的学习
到目前为止，我们进行了 word2vec 的改进。首先说明了 Embedding 层，又介绍了负采样的方法，然后对这两者进行了实现。现在我们进一步来实现进行了这些改进的神经网络，并在 PTB 数据集上进行学习，以获得更加实用的单词的分布式表示。

## CBOW模型的实现
这里，我们将改进上一章的简单的 `SimpleCBOW` 类，来实现 CBOW 模型。改进之处在于使用 Embedding 层和 `Negative Sampling Loss` 层。此外，我们将上下文部分扩展为可以处理任意的窗口大小。

改进版的 `CBOW` 类的实现如下所示。首先，我们来看一下初始化方法。

In [10]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 初始化权重
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 生成层
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # 使用Embedding层
            self.in_layers.append(layer) # 将所有的Embedding层保存在in_layers列表中
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5) # 负采样损失层

        # 将所有的权重和梯度整理到列表中
        layers = self.in_layers + [self.ns_loss] # 将所有层组合成一个列表
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 将单词的分布式表示设置为成员变量
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i]) # 从输入层获取词向量并相加
        h *= 1 / len(self.in_layers) # 中间层的输出取平均
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


这里的实现只是按适当的顺序调用各个层的正向传播（或反向传播），这是对上一章的 `SimpleCBOW` 类的自然扩展。不过，虽然 `forward(contexts, target)` 方法取的参数仍是上下文和目标词，但是它们是单词 ID 形式的（上一章中使用的是 one-hot 向量，不是单词 ID），具体示例如图所示。

<img src="./fig/context_example.png" alt="context_example" style="display: block; margin: 0 auto;">

图的右侧显示的单词 ID 列表是 `contexts` 和 `target` 的例子。可以看出，`contexts` 是一个二维数组，`target` 是一个一维数组，这样的数据被输入 `forward(contexts, target)` 中。以上就是 `CBOW` 类的说明。

## CBOW模型的学习代码
最后，我们来实现 CBOW 模型的学习部分。其实只是复用一下神经网络的学习，如下所示。

In [ ]:
import sys
sys.path.append('..') # 添加上级目录到系统路径中
from common import config
# 在用GPU运行时，请打开下面的注释（需要cupy）
# ===============================================
# config.GPU = True
# ===============================================
from common.np import *
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 设定超参数
window_size = 5 # 上下文窗口大小
hidden_size = 100 # 隐藏层大小
batch_size = 100 # 批量大小
max_epoch = 10 # 最大迭代轮数

# 读入数据
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id) # 词汇表大小

contexts, target = create_contexts_target(corpus, window_size) # 生成上下文和目标词
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target) # 将数据搬到GPU上

# 生成模型
model = CBOW(vocab_size, hidden_size, window_size, corpus) # CBOW模型
# model = SkipGram(vocab_size, hidden_size, window_size, corpus) # Skip-gram模型
optimizer = Adam() # Adam优化器
trainer = Trainer(model, optimizer) # 训练器

# 开始学习
trainer.fit(contexts, target, max_epoch, batch_size) # 训练模型
trainer.plot() # 绘制学习曲线

# 保存必要数据，以便后续使用
word_vecs = model.word_vecs # 获取词向量
if config.GPU:
    word_vecs = to_cpu(word_vecs) # 如果在GPU上运行，则将词向量搬回CPU
params = {}
params['word_vecs'] = word_vecs.astype(np.float16) # 将词向量转换为float16以节省空间
params['word_to_id'] = word_to_id # 保存单词到索引的映射
params['id_to_word'] = id_to_word # 保存索引到单词的映射
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl' # 保存文件名
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1) # 使用pickle保存数据

本次的 CBOW 模型的窗口大小为 5，隐藏层的神经元个数为 100。虽然具体取决于语料库的情况，但是一般而言，当窗口大小为 2 ~ 10、中间层的神经元个数（单词的分布式表示的维数）为 50 ~ 500 时，结果会比较好。稍后我们会对这些超参数进行讨论。

这次我们利用的 PTB 语料库比之前要大得多，因此学习需要很长时间（半天左右）。作为一种选择，我们提供了使用 GPU 运行的模式。如果要使用 GPU 运行，需要打开顶部的 “# config.GPU = True”。不过，使用 GPU 运行需要有一台安装了 NVIDIA GPU 和 CuPy 的机器。

在学习结束后，取出权重（输入侧的权重），并保存在文件中以备后用（用于单词和单词 ID 之间的转化的字典也一起保存）。这里，使用 Python 的 pickle 功能进行文件保存。pickle 可以将 Python 代码中的对象保存到文件中（或者从文件中读取对象）。

cbow_params.pkl 中提供了学习好的参数。如果不想等学习结束，可以使用本书提供的学习好的参数。根据学习环境的不同，学习到的权重数据也不一样。这是由权重初始化时用到的随机初始值、mini-batch 的随机选取，以及负采样的随机抽样造成的。因为这些随机性，最后得到的权重在各自的环境中会不一样。不过宏观来看，得到的结果（趋势）是类似的。

## CBOW模型的评价
现在，我们来评价一下上一节学习到的单词的分布式表示。这里我们使用第 2 章中实现的 `most_similar()` 函数，显示几个单词的最接近的单词。

In [11]:
import sys
sys.path.append('..') # 添加上级目录到系统路径中
from common.util import most_similar, analogy
import pickle

# 读入保存的模型参数
pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

# 加载数据
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 计算词向量的余弦相似度
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 类比任务
print('-'*50) # 分隔线
# analogy函数的参数依次为：单词A、单词B、单词C、word_to_id字典、id_to_word字典、词向量矩阵，表示 "A is to B as C is to ?"
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs) # 类比任务1
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs) # 类比任务2
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs) # 类比任务3
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs) # 类比任务4


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slow

我们看一下结果。首先，在查询 you 的情况下，近似单词中出现了人称代词 i（= I）和 we 等。接着，查询 year，可以看到 month、week 等表示时间区间的具有相同性质的单词。然后，查询 toyota，可以得到 ford、mazda 和 nissan 等表示汽车制造商的词汇。从这些结果可以看出，由 CBOW 模型获得的单词的分布式表示具有良好的性质。

此外，由 word2vec 获得的单词的分布式表示不仅可以将近似单词聚拢在一起，还可以捕获更复杂的模式，其中一个具有代表性的例子是因“king - man + woman = queen”而出名的类推问题（类比问题）。更准确地说，使用 word2vec 的单词的分布式表示，可以通过向量的加减法来解决类推问题。

如图所示，要解决类推问题，需要在单词向量空间上寻找尽可能使“man → woman”向量和“king →?”向量接近的单词。

<img src="./fig/analogy_task.png" alt="analogy_task" style="display: block; margin: 0 auto;">

结果符合我们的预期。第 1 个问题是 “king : man = queen :?”，这里正确地回答了 “woman”。第 2 个问题是 “take : took = go :?”，也按预期回答了 “went”。这是捕获了现在时和过去时之间的模式的证据，可以解释为单词的分布式表示编码了时态相关的信息。从第 3 题可知，单词的单数形式和复数形式之间的模式也被正确地捕获。可惜的是，对于第 4 题 “good : better = bad :?”，并没能回答出 “worse”。不过，看到 more、less 等比较级的单词出现在回答中，说明这些性质也被编码在了单词的分布式表示中。

像这样，使用 word2vec 获得的单词的分布式表示，可以通过向量的加减法求解类推问题。不仅限于单词的含义，它也捕获了语法中的模式。另外，我们还在 word2vec 的单词的分布式表示中发现了一些有趣的结果，比如 good 和 best 之间存在 better 这样的关系。

这里的类推问题的结果看上去非常好。不过遗憾的是，这是笔者特意选出来的能够被顺利解决的问题。实际上，很多问题都无法获得预期的结果。这是因为 PTB 数据集的规模还是比较小。如果使用更大规模的语料库，可以获得更准确、更可靠的单词的分布式表示，从而大大提高类推问题的准确率。

## word2vec相关的其他话题
关于 word2vec 的机制和实现，我们差不多都介绍完了。本节我们来讨论一些有关 word2vec 的其他话题。

## word2vec的应用例
使用 word2vec 获得的单词的分布式表示可以用来查找近似单词，但是单词的分布式表示的好处不仅仅在于此。在自然语言处理领域，单词的分布式表示之所以重要，原因就在于**迁移学习（transfer learning）**。迁移学习是指在某个领域学到的知识可以被应用于其他领域。

在解决自然语言处理任务时，一般不会使用 word2vec 从零开始学习单词的分布式表示，而是先在大规模语料库（Wikipedia、Google News 等文本数据）上学习，然后将学习好的分布式表示应用于某个单独的任务。比如，在文本分类、文本聚类、词性标注和情感分析等自然语言处理任务中，第一步的单词向量化工作就可以使用学习好的单词的分布式表示。在几乎所有类型的自然语言处理任务中，单词的分布式表示都有很好的效果！

单词的分布式表示的优点是可以将单词转化为固定长度的向量。另外，使用单词的分布式表示，也可以将文档（单词序列）转化为固定长度的向量。目前，关于如何将文档转化为固定长度的向量，相关研究已经进行了很多，最简单的方法是，把文档的各个单词转化为分布式表示，然后求它们的总和。这是一种被称为 bag-of-words 的不考虑单词顺序的模型（思想）。此外，使用即将在第 5 章中说明的循环神经网络，可以以更加优美的方式利用 word2vec 的单词的分布式表示来将文档转化为固定长度的向量。

将单词和文档转化为固定长度的向量是非常重要的。因为如果可以将自然语言转化为向量，就可以使用常规的机器学习方法（神经网络、SVM 等），如图所示。

<img src="./fig/progress.png" alt="progress" style="display: block; margin: 0 auto;">

由图可知，如果可以将自然语言书写的问题转化为固定长度的向量，就可以将这个向量作为其他机器学习系统的输入。通过将自然语言转化为向量，可以利用常规的机器学习框架输出目标答案（包括它的学习）。

在上图的流程中，单词的分布式表示的学习和机器学习系统的学习通常使用不同的数据集独立进行。比如，单词的分布式表示使用 Wikipedia 等通用语料库预先学习好，然后机器学习系统（SVM 等）再使用针对当前问题收集到的数据进行学习。但是，如果当前我们面对的问题存在大量的学习数据，则也可以考虑从零开始同时进行单词的分布式表示和机器学习系统的学习。

下面让我们结合具体的例子来说明一下单词的分布式表示的使用方法。假设你现在开发并维护着一个拥有超过 1 亿用户的智能手机应用，你的公司每天都要收到堆积如山的用户邮件（在 Twitter 等上也有很多吐槽）。虽然有一部意见是积极的，但是也存在很多表达不满的用户意见。

为此，你考虑开发一个可以对用户发来的邮件（吐槽等）自动进行分类的系统。如下图所示，你想根据邮件的内容将用户情感分为 3 类。如果可以正确地对用户情感进行分类，就可以按序浏览表达不满的用户邮件。如此一来，或许可以发现应用的致命问题，并尽早采取应对措施，从而提高用户的满意度。

<img src="./fig/email_example.png" alt="email_example" style="display: block; margin: 0 auto;">

要开发邮件自动分类系统，首先需要从收集数据（邮件）开始。在这个例子中，我们收集用户发送的邮件，并人工对邮件进行标注，打上表示 3 类情感的标签（positive/neutral/negative）。标注工作结束后，用学习好的 word2vec 将邮件转化为向量。然后，将向量化的邮件及其情感标签输入某个情感分类系统（SVM 或神经网络等）进行学习。

如本例所示，可以基于单词的分布式表示将自然语言处理问题转化为向量，这样就可以利用常规的机器学习方法来解决问题。另外，这样也能从 word2vec 的迁移学习中受益。换句话说，利用 word2vec 的单词的分布式表示，可以期待大多数自然语言处理任务获得精度上的提高。

## 单词向量的评价方法
使用 word2vec，我们得到了单词的分布式表示。那么，我们应该如何评价分布式表示的优劣呢？本节我们将简单说明分布式表示的评价方法。

正如上述情感分析的例子那样，在现实世界中，单词的分布式表示往往被用在具体的应用中。我们最终想要的是一个高精度的系统。这里我们必须考虑到的是，这个系统（比如情感分析系统）是由多个子系统组成的。所谓多个子系统，拿刚才的例子来说，包括生成单词的分布式表示的系统（word2vec）、对特定问题进行分类的系统（比如进行情感分类的 SVM 等）。

单词的分布式表示的学习和分类系统的学习有时可能会分开进行。在这种情况下，如果要调查单词的分布式表示的维数如何影响最终的精度，首先需要进行单词的分布式表示的学习，然后再利用这个分布式表示进行另一个机器学习系统的学习。换句话说，在进行两个阶段的学习之后，才能进行评价。在这种情况下，由于需要调试出对两个系统都最优的超参数，所以非常费时。

因此，单词的分布式表示的评价往往与实际应用分开进行。此时，经常使用的评价指标有“相似度”和“类推问题”。

单词相似度的评价通常使用人工创建的单词相似度评价集来评估。比如，cat 和 animal 的相似度是 8，cat 和 car 的相似度是 2……类似这样，用 0 ~ 10 的分数人工地对单词之间的相似度打分。然后，比较人给出的分数和 word2vec 给出的余弦相似度，考察它们之间的相关性。

类推问题的评价是指，基于诸如 “king : queen = man :?” 这样的类推问题，根据正确率测量单词的分布式表示的优劣。比如，论文中给出了一个类推问题的评价结果，其部分内容如图所示。

<img src="./fig/evaluation_result.png" alt="evaluation_result" style="display: block; margin: 0 auto;">

在图中，以 word2vec 的模型、单词的分布式表示的维数和语料库的大小为参数进行了比较实验，结果在右侧的 3 列中。上图的 Semantics 列显示的是推断单词含义的类推问题（像 “king : queen = actor : actress” 这样询问单词含义的问题）的正确率，Syntax 列是询问单词形态信息的问题，比如 “bad : worst = good : best”。

由图可知：
- 模型不同，精度不同（根据语料库选择最佳的模型）
- 语料库越大，结果越好（始终需要大数据）
- 单词向量的维数必须适中（太大会导致精度变差）

基于类推问题可以在一定程度上衡量“是否正确理解了单词含义或语法问题”。因此，在自然语言处理的应用中，能够高精度地解决类推问题的单词的分布式表示应该可以获得好的结果。但是，单词的分布式表示的优劣对目标应用贡献多少（或者有无贡献），取决于待处理问题的具体情况，比如应用的类型或语料库的内容等。也就是说，不能保证类推问题的评价高，目标应用的结果就一定好。这一点请一定注意。

## 小结 
- Embedding 层保存单词的分布式表示，在正向传播时，提取单词 ID 对应的向量
- 因为 word2vec 的计算量会随着词汇量的增加而成比例地增加，所以最好使用近似计算来加速
- 负采样技术采样若干负例，使用这一方法可以将多分类问题转化为二分类问题进行处理
- 基于 word2vec 获得的单词的分布式表示内嵌了单词含义，在相似的上下文中使用的单词在单词向量空间上处于相近的位置
- word2vec 的单词的分布式表示的一个特性是可以基于向量的加减法运算来求解类推问题
- word2vec 的迁移学习能力非常重要，它的单词的分布式表示可以应用于各种各样的自然语言处理任务